## 1. Đọc các bảng từ url (dti từ 2022 đến 2024)

In [2]:
import pandas as pd
import os
from urllib.parse import urlparse

In [3]:
# Danh sách URL
urls = [
    "https://dti.gov.vn/xep-hang-2022",
    "https://dti.gov.vn/xep-hang-2023-all",
    "https://dti.gov.vn/xep-hang-2024-63-tinh",  # 2024 63 tỉnh
    "https://dti.gov.vn/xep-hang-2024"           # 2024 34 tỉnh
]

# Thư mục lưu kết quả
output_folder = r"D:\Study\OpenDX\data\dti"

# Tạo thư mục chính nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

for url in urls:
    print(f"\nĐang xử lý URL: {url}")

    # Đọc tất cả bảng từ trang
    try:
        tables = pd.read_html(url)
    except Exception as e:
        print(f"Không thể đọc bảng từ {url}: {e}")
        continue

    print(f"Số bảng tìm được: {len(tables)}")

    # Tạo tên folder con dựa trên phần cuối của URL
    url_path = urlparse(url).path.strip("/").replace("/", "_")
    folder_path = os.path.join(output_folder, url_path)
    os.makedirs(folder_path, exist_ok=True)

    # Lưu từng bảng ra JSON trong folder con
    for i, df in enumerate(tables):
        filename = os.path.join(folder_path, f"table_{i}.json")
        df.to_json(filename, orient="records", force_ascii=False)
        print(f"Đã lưu: {filename}")

    # Hiển thị thử vài bảng đầu
    # for i, df in enumerate(tables):
    #     print(f"\n===== Bảng {i} =====")
    #     print(df.head())



Đang xử lý URL: https://dti.gov.vn/xep-hang-2022
Số bảng tìm được: 9
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_0.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_1.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_2.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_3.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_4.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_5.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_6.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_7.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2022\table_8.json

Đang xử lý URL: https://dti.gov.vn/xep-hang-2023-all
Số bảng tìm được: 9
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_0.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_1.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_2.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_3.json
Đã lưu: D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_4.j

## 2. Tạo các file dti_map.json tương ứng để vẽ lên bản đồ

In [4]:
import json
import json
import pandas as pd
import re
from pathlib import Path

In [20]:
import json
import pandas as pd
import re
from pathlib import Path

def process_dti_table(
    table_json_path: str,        # Đường dẫn đến table JSON gốc
    geojson_path: str,           # Đường dẫn đến file geojson
    output_json_path: str        # Đường dẫn lưu file JSON kết quả cuối cùng
):
    """
    Xử lý DTI table JSON:
    - Tạo df từ file JSON
    - Lấy df2 với các cột province và pdti (linh hoạt với tên cột khác nhau)
    - Chuẩn hóa tên tỉnh
    - Mapping vào geojson
    - Lưu kết quả vào file JSON cuối cùng
    """
    # ---- Load table JSON ----
    with open(table_json_path, 'r', encoding='utf-8') as f:
        dti_table = json.load(f)

    df = pd.DataFrame(dti_table)

    # ---- Xác định cột PDti ----
    possible_pdti_cols = [
        "CHUYỂN ĐỔI SỐ CẤP TỈNH (PDTI)",
        "Bộ chỉ số dành cho cấp Tỉnh"
    ]
    pdti_col = None
    for col in possible_pdti_cols:
        if col in df.columns:
            pdti_col = col
            break
    if pdti_col is None:
        raise ValueError(f"Không tìm thấy cột PDti trong bảng, kiểm tra các cột: {possible_pdti_cols}")

    # ---- Tạo df2 với cột cần thiết ----
    df2 = df[['Các tỉnh, thành phố trực thuộc TW', pdti_col]].copy()
    df2.columns = ['province', 'pdti']
    df2['province'] = df2['province'].str.normalize('NFKC').str.replace('TP. ', '', regex=False)

    # ---- Hàm chuẩn hóa tên tỉnh ----
    def normalize(name: str):
        name = name.strip()
        name = re.sub(r"^(Tỉnh|Thành phố)\s+", "", name, flags=re.IGNORECASE)
        name = re.sub(r"\s+", " ", name)
        return name

    # ---- Mapping tên thủ công ----
    manual_map = {
        "Hồ Chí Minh": "TP. Hồ Chí Minh",
        "Huế": "Thừa Thiên Huế",
        "Hòa Bình": "Hoà Bình",
    }

    # ---- Load geojson ----
    with open(geojson_path, 'r', encoding='utf-8') as f:
        geo = json.load(f)

    geo_names = {normalize(ft["properties"]["ten_tinh"]): ft["properties"]["ten_tinh"] for ft in geo["features"]}

    # ---- Mapping tên và tạo output ----
    output = []
    missing = []

    for _, row in df2.iterrows():
        p_raw = row["province"]

        if p_raw in manual_map:
            output.append({
                "province": manual_map[p_raw],
                "pdti": row["pdti"]
            })
            continue

        p_norm = normalize(p_raw)
        if p_norm in geo_names:
            output.append({
                "province": geo_names[p_norm],
                "pdti": row["pdti"]
            })
        else:
            missing.append(p_raw)

    # ---- Lưu file JSON kết quả ----
    output_path = Path(output_json_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    # ---- Thông báo ----
    if missing:
        print("Không match:", missing)
    else:
        print("Tất cả tỉnh đều match 100%")
    
    print(f"Đã lưu file kết quả: {output_path}")


### dti 2022

In [21]:
process_dti_table(
    table_json_path=r"D:\Study\OpenDX\data\dti\xep-hang-2022\table_1.json",
    geojson_path=r"D:\Study\OpenDX\frontend\public\VietNam_tinhthanh.geojson",
    output_json_path=r"D:\Study\OpenDX\frontend\public\dti_2022_map.json"
)


Tất cả tỉnh đều match 100%
Đã lưu file kết quả: D:\Study\OpenDX\frontend\public\dti_2022_map.json


### dti 2023

In [22]:
with open(r"D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_1.json", 'r', encoding='utf-8') as f:
        dti_table_2023 = json.load(f)
df_2023 = pd.DataFrame(dti_table_2023)
df_2023 

,Xếp hạng DTI,"Các tỉnh, thành phố trực thuộc TW",Chỉ số Nhận thức số,Chỉ số Thể chế số,Chỉ số Hạ tầng số,Chỉ số Nhân lực số,Chỉ số An toàn thông tin mạng,Chỉ số Hoạt động chính quyền số,Chỉ số Hoạt động kinh tế số,Chỉ số Hoạt động xã hội số,CHUYỂN ĐỔI SỐ CẤP TỈNH (PDTI)
0,1,TP. Đà Nẵng,1.0000,1.0,0.8362,0.8476,0.9267,0.8790,0.7262,0.5881,0.8340
1,2,TP. Hồ Chí Minh,0.9857,0.9,0.8364,0.7703,0.7725,0.8643,0.8190,0.5319,0.8020
2,3,TP. Huế,0.9714,1.0,0.7098,0.7062,0.9167,0.8050,0.7589,0.4048,0.7660
3,4,Lạng Sơn,1.0000,0.8,0.7336,0.8091,0.7893,0.8489,0.6801,0.5005,0.7601
4,5,TP. Cần Thơ,0.9857,0.8,0.7884,0.7431,0.8800,0.8055,0.6452,0.4617,0.7468
...,...,...,...,...,...,...,...,...,...,...,...
58,59,An Giang,0.9214,0.6,0.5230,0.5797,0.3760,0.5869,0.5834,0.3553,0.5582
59,59,Phú Yên,0.8643,0.9,0.6059,0.6550,0.3492,0.4558,0.5315,0.3325,0.5582
60,61,Quảng Trị,0.8833,0.8,0.6511,0.4467,0.4520,0.5460,0.4693,0.3058,0.5488
61,62,Cao Bằng,0.7857,0.8,0.5274,0.4640,0.4555,0.4424,0.5495,0.3381,0.5249


In [23]:
process_dti_table(
    table_json_path=r"D:\Study\OpenDX\data\dti\xep-hang-2023-all\table_1.json",
    geojson_path=r"D:\Study\OpenDX\frontend\public\VietNam_tinhthanh.geojson",
    output_json_path=r"D:\Study\OpenDX\frontend\public\dti_2023_map.json"
)


Tất cả tỉnh đều match 100%
Đã lưu file kết quả: D:\Study\OpenDX\frontend\public\dti_2023_map.json


### dti 2024 63 tỉnh

In [24]:
process_dti_table(
    table_json_path=r"D:\Study\OpenDX\data\dti\xep-hang-2024-63-tinh\table_0.json",
    geojson_path=r"D:\Study\OpenDX\frontend\public\VietNam_tinhthanh.geojson",
    output_json_path=r"D:\Study\OpenDX\frontend\public\dti_2024_63_tinh_map.json"
)


Tất cả tỉnh đều match 100%
Đã lưu file kết quả: D:\Study\OpenDX\frontend\public\dti_2024_63_tinh_map.json


## 3. Bản đồ mới 34 tỉnh thành

In [1]:
import json

geojson_file = r"D:\Study\OpenDX\frontend\public\VietNam_tinhthanh_34.geojson"  # đổi tên nếu khác

with open(geojson_file, "r", encoding="utf-8") as f:
    geo = json.load(f)

province_names = []

for feature in geo["features"]:
    props = feature.get("properties", {})
    name = props.get("ten_tinh")  # lấy tên tỉnh đúng theo cấu trúc
    if name:
        province_names.append(name)

province_names = sorted(set(province_names))

print("Số lượng tỉnh tìm được:", len(province_names))
for p in province_names:
    print(p)

Số lượng tỉnh tìm được: 34
An Giang
Bắc Ninh
Cao Bằng
Cà Mau
Cần Thơ
Gia Lai
Huế
Hà Nội
Hà Tĩnh
Hưng Yên
Hải Phòng
Khánh Hòa
Lai Châu
Lào Cai
Lâm Đồng
Lạng Sơn
Nghệ An
Ninh Bình
Phú Thọ
Quảng Ngãi
Quảng Ninh
Quảng Trị
Sơn La
TP. Hồ Chí Minh
Thanh Hóa
Thái Nguyên
Tuyên Quang
Tây Ninh
Vĩnh Long
Điện Biên
Đà Nẵng
Đắk Lắk
Đồng Nai
Đồng Tháp


In [5]:
with open(r"D:\Study\OpenDX\data\dti\xep-hang-2024\table_1.json", 'r', encoding='utf-8') as f:
        dti_table_2023 = json.load(f)
df_2023 = pd.DataFrame(dti_table_2023)
df_2023 

,Xếp hạng DTI,"Các tỉnh, thành phố trực thuộc TW",Chỉ số Nhận thức số,Chỉ số Thể chế số,Chỉ số Hạ tầng số,Chỉ số Nhân lực số,Chỉ số An toàn thông tin mạng,Chỉ số Hoạt động chính quyền số,Chỉ số Hoạt động kinh tế số,Chỉ số Hoạt động xã hội số,Bộ chỉ số dành cho cấp Tỉnh
0,1,TP. Hà Nội,1.0000,1.0000,0.9286,0.9000,0.8082,0.5661,0.9250,0.8127,0.8241
1,2,TP. Huế,1.0000,1.0000,0.9664,0.8075,1.0000,0.7519,0.5617,0.6451,0.7951
2,3,TP. Hải Phòng,1.0000,1.0000,0.9898,0.9320,0.7131,0.6375,0.7613,0.6253,0.7857
3,4,TP. Hồ Chí Minh,1.0000,1.0000,0.9881,0.9165,0.7168,0.5513,0.7538,0.7024,0.7770
4,5,Thanh Hóa,1.0000,1.0000,0.9544,1.0000,0.9500,0.6306,0.6179,0.5359,0.7742
5,6,Điện Biên,0.9000,1.0000,0.9466,0.9846,0.9722,0.6458,0.5487,0.5637,0.7651
6,7,Hà Tĩnh,1.0000,1.0000,0.7529,1.0000,1.0000,0.6331,0.7199,0.5139,0.7629
7,8,Thái Nguyên,1.0000,1.0000,0.8738,0.8911,0.7873,0.7809,0.5651,0.5059,0.7521
8,9,Quảng Ninh,1.0000,1.0000,1.0000,0.9000,0.9152,0.4088,0.7325,0.6025,0.7484
9,10,Lạng Sơn,1.0000,1.0000,1.0000,0.9000,1.0000,0.5569,0.7101,0.3685,0.7480


In [6]:
import json
import pandas as pd
import re
from pathlib import Path

def process_dti_table_34(
    table_json_path: str,        # Đường dẫn đến table JSON gốc
    geojson_path: str,           # Đường dẫn đến file geojson
    output_json_path: str        # Đường dẫn lưu file JSON kết quả cuối cùng
):
    """
    Xử lý DTI table JSON:
    - Tạo df từ file JSON
    - Lấy df2 với các cột province và pdti (linh hoạt với tên cột khác nhau)
    - Chuẩn hóa tên tỉnh
    - Mapping vào geojson
    - Lưu kết quả vào file JSON cuối cùng
    """
    # ---- Load table JSON ----
    with open(table_json_path, 'r', encoding='utf-8') as f:
        dti_table = json.load(f)

    df = pd.DataFrame(dti_table)

    # ---- Xác định cột PDti ----
    possible_pdti_cols = [
        "CHUYỂN ĐỔI SỐ CẤP TỈNH (PDTI)",
        "Bộ chỉ số dành cho cấp Tỉnh"
    ]
    pdti_col = None
    for col in possible_pdti_cols:
        if col in df.columns:
            pdti_col = col
            break
    if pdti_col is None:
        raise ValueError(f"Không tìm thấy cột PDti trong bảng, kiểm tra các cột: {possible_pdti_cols}")

    # ---- Tạo df2 với cột cần thiết ----
    df2 = df[['Các tỉnh, thành phố trực thuộc TW', pdti_col]].copy()
    df2.columns = ['province', 'pdti']
    df2['province'] = df2['province'].str.normalize('NFKC').str.replace('TP. ', '', regex=False)

    # ---- Hàm chuẩn hóa tên tỉnh ----
    def normalize(name: str):
        name = name.strip()
        name = re.sub(r"^(Tỉnh|Thành phố)\s+", "", name, flags=re.IGNORECASE)
        name = re.sub(r"\s+", " ", name)
        return name

    # ---- Mapping tên thủ công ----
    manual_map = {
        "Hồ Chí Minh": "TP. Hồ Chí Minh",
    }

    # ---- Load geojson ----
    with open(geojson_path, 'r', encoding='utf-8') as f:
        geo = json.load(f)

    geo_names = {normalize(ft["properties"]["ten_tinh"]): ft["properties"]["ten_tinh"] for ft in geo["features"]}

    # ---- Mapping tên và tạo output ----
    output = []
    missing = []

    for _, row in df2.iterrows():
        p_raw = row["province"]

        if p_raw in manual_map:
            output.append({
                "province": manual_map[p_raw],
                "pdti": row["pdti"]
            })
            continue

        p_norm = normalize(p_raw)
        if p_norm in geo_names:
            output.append({
                "province": geo_names[p_norm],
                "pdti": row["pdti"]
            })
        else:
            missing.append(p_raw)

    # ---- Lưu file JSON kết quả ----
    output_path = Path(output_json_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    # ---- Thông báo ----
    if missing:
        print("Không match:", missing)
    else:
        print("Tất cả tỉnh đều match 100%")
    
    print(f"Đã lưu file kết quả: {output_path}")


In [8]:
process_dti_table_34(
    table_json_path=r"D:\Study\OpenDX\data\dti\xep-hang-2024\table_1.json",
    geojson_path=r"D:\Study\OpenDX\frontend\public\VietNam_tinhthanh_34.geojson",
    output_json_path=r"D:\Study\OpenDX\frontend\public\dti_2024_34_tinh_map.json"
)


Tất cả tỉnh đều match 100%
Đã lưu file kết quả: D:\Study\OpenDX\frontend\public\dti_2024_34_tinh_map.json
